## Dependencias

In [ ]:
pip install undetected-chromedriver

## Funciones genericas

### Definicion de mis user agent

In [ ]:
import random
import time
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import undetected_chromedriver.v2 as uc2
import json
import csv
from selenium import webdriver

user_agents = ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.53 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
            'Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0',
            'Mozilla/5.0 (Linux; Android 10; LM-Q710(FGN)) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 10; SM-N960U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 10) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 10; HD1913) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.101 Mobile Safari/537.36 EdgA/97.0.1072.69',
            'Mozilla/5.0 (Windows Mobile 10; Android 10.0; Microsoft; Lumia 950XL) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Mobile Safari/537.36 Edge/40.15254.603',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; Xbox; Xbox One) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36 Edge/44.18363.8131',
            'Mozilla/5.0 (CrKey armv7l 1.5.16041) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.0 Safari/537.36',
            'Mozilla/5.0 (Linux; Android 5.1.1; AFTT Build/LVY48F; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/49.0.2623.10',
            'Dalvik/2.1.0 (Linux; U; Android 5.1; AFTM Build/LMY47O)',
            'Mozilla/5.0 (Linux; Tizen 2.3) AppleWebKit/538.1 (KHTML, like Gecko)Version/2.3 TV Safari/538.1',
            'Mozilla/5.0 (Web0S; Linux/SmartTV) AppleWebKit/537.36 (KHTML, like Gecko) QtWebEngine/5.2.1 Chr0me/38.0.2125.122 Safari/537.36 LG Browser/8.00.00(LGE; 60UH6550-UB; 03.00.15; 1; DTV_W16N); webOS.TV-2016; LG NetCast.TV-2013 Compatible (LGE, 60UH6550-UB, wireless)',
            'Mozilla/5.0 (Linux; Android 7.0; SM-G892A Build/NRD90M; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/60.0.3112.107 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 6.0.1; SM-G935S Build/MMB29K; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/55.0.2883.91 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 6.0.1; E6653 Build/32.2.A.0.253) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.36',
            'Mozilla/5.0 (Linux; Android 6.0; HTC One M9 Build/MRA58K) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.98 Mobile Safari/537.3'
]
agents = len(user_agents) - 1

### Definicion de funciones para busqueda de valores

In [ ]:
def get_only_fisrt_value(keys_and_values):
    for key in keys_and_values:
        datas = keys_and_values[key]
        if len(datas) == 0:
            keys_and_values[key] = ''
        else:
            keys_and_values[key] = keys_and_values[key][0]
    return keys_and_values

def search_all_coincidences(key, path, data_json):
    coincidences = []
    data_json = get_sub_path(path, data_json)
    if isinstance(data_json, list):
        for data in data_json:
            if isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    else:
        for data_key in data_json:
            data = data_json[data_key]
            if key == data_key:
                coincidences.append(data)
            elif isinstance(data, dict) or isinstance(data, list):
                coincidences = coincidences + search_all_coincidences(key, None, data)
    return coincidences

def get_sub_path(path, data_json):
    if path != None:
        path = path.split('/')
        for sub_path in path:
            data_json = data_json[sub_path]
    return data_json

### Funcion para escritura de resultados

In [ ]:
def write_in_file(page_cont, rows, keys_names):
    file_name = "./../generated/scrapy/cars/data" + str(page_cont) + ".csv"
    with open(file_name, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys_names)
        writer.writeheader()
        for data in rows:
            writer.writerow(data)

### Funciones para scrapy

In [ ]:
def get_path(keys, key):
    path = None
    if isinstance(keys, dict):
        path = keys[key]
    return path

def download_url(driver, page, rows):
    driver.get(page)
    page_source = driver.page_source
    if page not in pages_without_interest:
        initialPos = page_source.find('window.__INITIAL_PROPS__ = JSON.parse(')
        lastPos = len(page_source)
        page_source = page_source[initialPos : lastPos]
        initialPos = 39
        lastPos = page_source.find(');</script>') - 1
        page_source = page_source[initialPos : lastPos]
        page_source=page_source.replace('\\\\\\\"', '')
        page_source=page_source.replace('\\\"', '\"')
        data_json = json.loads(page_source)

        # Prepare datas
        keys_and_values = {}
        for key in keys:
            path = get_path(keys, key)
            keys_and_values
            keys_and_values[key] = search_all_coincidences(key, path, data_json)
        keys_and_values = get_only_fisrt_value(keys_and_values)
        rows.append(keys_and_values)
    else:
        print("Pagina sin interes: " + page)

In [ ]:
class BotException(Exception):
    pass

def download_datas(driver, pages):
    
    rows = []
    
    for i in range(len(pages)):
        page = pages[i]
        
        try:
            download_url(driver, page, rows)
        except Exception as e:
            print("Hubo algun problema con la url: " + page)
            print(e)
        
        page_source = driver.page_source
        if "Algo en tu navegador nos hizo pensar que eres un bot" in page_source:
            while True:
                try:
                    download_url(driver, page, rows)
                    break
                except Exception as e:
                    print("Hubo algun problema con la url: " + page)
                    print(e)
                print("Esperando resolucion de captcha")
                time.sleep(30)
                print("Probando de nuevo")
    
    return rows

## Parametros y creacion de lista de urls para scrapy

In [ ]:
keys = {'color': 'ad', 'fuelTypeId': 'ad', 'id': 'ad', 'km': 'ad', 'make': 'ad', 'makeId': 'ad', 'province': 'ad', 'provinceId': 'ad', 'title': 'ad', 'transmissionType': 'ad', 'transmissionTypeId': 'ad', 'year': 'ad'
        , 'brand': 'leadData', 'brand_id': 'leadData', 'fuel_types': 'ad', 'model': 'leadData', 'model_id': 'leadData', 'price': 'leadData'
        , 'seller_type': 'leadData', 'bodyType': 'vehicleInfo/specifications', 'cubicCapacity': 'vehicleInfo/specifications', 'doors': 'vehicleInfo/specifications'
        , 'hp': 'vehicleInfo/specifications'}
keys = {'color': 'ad','fuelTypeId': 'ad','id': 'ad','km': 'ad','make': 'ad','makeId': 'ad','province': 'ad','provinceId': 'ad','title': 'ad','transmissionType': 'ad','transmissionTypeId': 'ad','year': 'ad','brand': 'leadData','brand_id': 'leadData','fuel_types': 'ad','model': 'leadData','model_id': 'leadData','price': 'leadData','seller_type': 'leadData','bodyType': 'vehicleInfo/specifications','cubicCapacity': 'vehicleInfo/specifications','doors': 'vehicleInfo/specifications','hp': 'vehicleInfo/specifications','acceleration': 'vehicleInfo/specifications','consumptionAverage': 'vehicleInfo/specifications','consumptionExtraUrban': 'vehicleInfo/specifications','consumptionUrban': 'vehicleInfo/specifications','emissions': 'vehicleInfo/specifications','fuelTankCapacity': 'vehicleInfo/specifications','height': 'vehicleInfo/specifications','length': 'vehicleInfo/specifications','luggageCapacity': 'vehicleInfo/specifications','manufacturerPrice': 'vehicleInfo/specifications','seats': 'vehicleInfo/specifications','topSpeed': 'vehicleInfo/specifications','weight': 'vehicleInfo/specifications','width': 'vehicleInfo/specifications','seller_type': 'pageTrackingData'}
keys_names = [ key for key in keys.keys() ]

## Ejecucion

### Creacion de driver chrome

In [ ]:
def get_driver(page_cont):
    patcher = uc.Patcher()
    patcher.auto()
    user_agent_for_page = user_agents[page_cont % agents]
    options = webdriver.ChromeOptions() 
    # options.headless = True
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument(user_agent_for_page)
    driver = uc.Chrome(options=options, executable_path=patcher.executable_path)
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent_for_page})
    return driver

### Scrapy

In [ ]:
urls_range, num_file = range(0, 1), 0

pages_without_interest = [ 'https://www.coches.net', 'https://www.coches.net/segunda-mano' ]
urls, url_files = [], []

for num in urls_range:
    f_r = open('./../generated/scrapy/urls/car_liks' + str(num) + '.json', "r")
    urls.append(json.loads(f_r.read()))
    f_r.close()

for page in urls:
    new_page = pages_without_interest[:]
    for url in page:
        new_page.append('https://www.coches.net' + url)
    url_files.append(new_page)

driver, all_rows, proccessed_pages, size_pages = get_driver(num_file), [], 0, len(url_files)

for pages in url_files:
    all_rows = all_rows + download_datas(driver, pages)
    proccessed_pages = proccessed_pages + 1
    print("Paginas procesadas: {}/{}".format(proccessed_pages, size_pages))

write_in_file(num_file, all_rows, keys_names)